<a href="https://colab.research.google.com/github/geersenthil/Topic-Modeling-/blob/main/LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import required libraries

In [1]:
import nltk 
nltk.download("stopwords")
!pip install pyLDAvis

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     |████████████████████████████████| 1.7 MB 3.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=5b0933ac2d3f32d9c2f82cd77ead1df61097e5c0e66f41f4b40431b62ade2c64
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [79]:
import re
import pandas as pd
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.parsing import strip_tags, strip_numeric, strip_multiple_whitespaces, stem_text, strip_punctuation, remove_stopwords
from gensim.parsing import preprocess_string

#Dataset
from sklearn.datasets import fetch_20newsgroups

#Spacy
import spacy
from nltk.corpus import stopwords

#Visuals
import pyLDAvis
import pyLDAvis.gensim_models

#warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [80]:
stopwords = stopwords.words("english")

Load Data into Dataframe

In [81]:
news_group = fetch_20newsgroups()

news_group_data = news_group.data
news_group_target_names = news_group.target_names
news_group_target = news_group.target

In [82]:
news_df = pd.DataFrame({'news': news_group_data})

In [83]:
news_data = news_df['news'].sample(100)

Pre-process data

Lemmatization to get base words

In [84]:
def lemmatization(texts, allowed_postags=["NOUN","ADJ","VERB","ADV"]):
  nlp = spacy.load('en_core_web_sm', disable=["parser", "ner"])
  texts_out = []
  for text in texts:
    doc = nlp(text)
    new_text = []
    for token in doc:
        if token.pos_ in allowed_postags:
          new_text.append(token.lemma_)
    final = " ".join(new_text)
    texts_out.append(final)
  return (texts_out)
 

In [85]:

lemmatizated_text = lemmatization(news_data)


In [86]:
print(lemmatizated_text[0:500])


['subject enclose kmcvay@oneb.almanac.bc.ca organization article write seem yet messenger get kill message carry say seem bar evidence contrary simply deranged fanatic think neccessary take whole bunch folk child satisfy delusional mania circa mean time sure learn lot evil corruption surprised thing get rotten nope fruitcake demonstrate such evil corruption century v32 xenix serve public access home', 'subject when go ship originator reader line write external already ship ship quite awhile now demand unit high pretty rare get cd drive also hear rumor bundle couple can confirm indeed cd bundle usually get cd application game photo compile list post will post update version list rsn :)', 'palmer@cco.caltech.edu subject boom posting host alumni.caltech.edu write article write hear insert favorite rumor here will supposedly coincide even bad city proposal rent space orbit billboard consider caliber people run city tell go leer orbit would just like point much easy place object orbital alt

In [87]:
#tokenize
def gen_words(texts):
  final = []
  for text in texts:
    new = gensim.utils.simple_preprocess(text, deacc=True)
    final.append(new)
  return (final)

data_word = gen_words(lemmatizated_text)

In [88]:
print(data_word[0][0:20])

['subject', 'enclose', 'kmcvay', 'oneb', 'almanac', 'bc', 'ca', 'organization', 'article', 'write', 'seem', 'yet', 'messenger', 'get', 'kill', 'message', 'carry', 'say', 'seem', 'bar']


In [89]:
#id2word

id2word = corpora.Dictionary(data_word)
corpus = []
for text in data_word:
  new = id2word.doc2bow(text)
  corpus.append(new)

#id and frequency of word
print (corpus[0][0:20])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1)]


In [90]:
word = id2word[[0][:1][0]]
print(word)

access


In [91]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[:-1], id2word = id2word, num_topics = 20, random_state = 100, update_every=1, chunksize=100, passes = 10, alpha="auto")

In [78]:
test_doc = corpus[-1]
vector = lda_model[test_doc]
print(vector)

# sort by match f
def Sort(sub_li):
  sub_li.sort(key = lambda x: x[1])
  sub_li.reverse()
  return(sub_li)
new_vector = Sort(vector)
print(new_vector)



[(1, 0.045874346), (10, 0.10753278), (11, 0.5649328), (12, 0.2254055), (19, 0.043226607)]
[(11, 0.5649328), (12, 0.2254055), (10, 0.10753278), (1, 0.045874346), (19, 0.043226607)]


Visualise Data

In [92]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=10)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15    -0.215577 -0.078764       1        1  13.124097
8     -0.258790  0.015900       2        1  10.429310
17    -0.127370 -0.124944       3        1   9.245536
9     -0.028528 -0.257222       4        1   7.556461
18    -0.052429  0.263784       5        1   7.470345
1     -0.194446  0.108897       6        1   7.141718
2     -0.139776  0.187343       7        1   5.750850
14    -0.167422 -0.205448       8        1   5.144248
7      0.077298 -0.211255       9        1   4.489941
3      0.175050 -0.163336      10        1   4.389049
19     0.047997 -0.130386      11        1   4.220974
11     0.194897  0.158068      12        1   3.816906
10    -0.081867  0.042826      13        1   3.161221
12     0.043078 -0.053810      14        1   3.130187
0     -0.004521  0.147725      15        1   2.348015
13     0.110222  0.212930      16        1   2.343077
5      0.242616 -0.047258      17        1   2.119947
4      0.200115  0.020542      18        1   2.066326
16     0.132762  0.075899      19        1   1.740516
6      0.046689  0.038508      20        1   0.311275, topic_info=         Term       Freq      Total Category  logprob  loglift
1269      bus  10.000000  10.000000  Default  10.0000  10.0000
91       will  75.000000  75.000000  Default   9.0000   9.0000
191       use  48.000000  48.000000  Default   8.0000   8.0000
400     claim  18.000000  18.000000  Default   7.0000   7.0000
747       key  20.000000  20.000000  Default   6.0000   6.0000
...       ...        ...        ...      ...      ...      ...
43    subject   0.181804  79.298277  Topic20  -5.1329  -0.3058
2     article   0.181608  54.776962  Topic20  -5.1339   0.0631
189      tell   0.181475  23.305151  Topic20  -5.1347   0.9169
137      just   0.181213  45.470592  Topic20  -5.1361   0.2471
114     could   0.181105  23.847265  Topic20  -5.1367   0.8919

[433 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
1380      4  0.627262       accept
1380      9  0.209087       accept
1935     19  0.863931     accutane
1314     19  0.863210         acid
1528      4  0.933135  acknowledge
...     ...       ...          ...
197      13  0.048798         year
197      14  0.048798         year
197      16  0.146393         year
197      19  0.048798         year
1344     19  0.884494         zinc

[1170 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 9, 18, 10, 19, 2, 3, 15, 8, 4, 20, 12, 11, 13, 1, 14, 6, 5, 17, 7])